In [29]:
import numpy as np 
import pandas as pd
import pickle

In [30]:
df = pd.read_csv('/Users/home/Documents/GitHub/FormulaAIHacks/weather.csv',low_memory=False)

In [31]:
df = df[(df['M_NUM_WEATHER_FORECAST_SAMPLES'] != 0) & (df['M_WEATHER_FORECAST_SAMPLES_M_SESSION_TYPE'] != 0)]

In [32]:
df = df.drop([
    'M_PACKET_FORMAT', 
    'M_GAME_MAJOR_VERSION',  
    'M_GAME_MINOR_VERSION',  
    'M_PACKET_VERSION', 
    'M_PACKET_ID', 
    'M_SECONDARY_PLAYER_CAR_INDEX',
    'M_SLI_PRO_NATIVE_SUPPORT', 
    'Unnamed: 58',
    'GAMEHOST',
    'M_SESSION_UID',
    'M_SESSION_TIME',
    'M_FRAME_IDENTIFIER',
], axis=1)

In [33]:
pd.to_datetime(df['TIMESTAMP'], unit='s')

17017     2022-01-21 00:16:50
17018     2022-01-21 00:16:50
17019     2022-01-21 00:16:50
17020     2022-01-21 00:16:50
17021     2022-01-21 00:16:50
                  ...        
3572287   2022-01-15 23:14:51
3572288   2022-01-15 23:14:51
3572289   2022-01-15 23:14:51
3572290   2022-01-15 23:14:51
3572291   2022-01-15 23:14:51
Name: TIMESTAMP, Length: 1393074, dtype: datetime64[ns]

In [34]:
df_subset = df[['TIMESTAMP', 'M_AIR_TEMPERATURE'] + df.columns.tolist()[31:39]]
'rows with na:', df_subset.shape[0] - df_subset.dropna().shape[0]

('rows with na:', 748671)

In [35]:
df_subset

,TIMESTAMP,M_AIR_TEMPERATURE,M_WEATHER_FORECAST_SAMPLES_M_WEATHER,M_WEATHER_FORECAST_SAMPLES_M_TRACK_TEMPERATURE,M_TRACK_TEMPERATURE_CHANGE,M_WEATHER_FORECAST_SAMPLES_M_AIR_TEMPERATURE,M_AIR_TEMPERATURE_CHANGE,M_RAIN_PERCENTAGE,M_WEATHER,M_AI_DIFFICULTY
17017,1.642724e+09,25,NaN,NaN,NaN,NaN,NaN,NaN,0,31
17018,1.642724e+09,25,NaN,NaN,NaN,NaN,NaN,NaN,0,31
17019,1.642724e+09,25,NaN,NaN,NaN,NaN,NaN,NaN,0,31
17020,1.642724e+09,25,NaN,NaN,NaN,NaN,NaN,NaN,0,31
17021,1.642724e+09,25,NaN,NaN,NaN,NaN,NaN,NaN,0,31
...,...,...,...,...,...,...,...,...,...,...
3572287,1.642288e+09,25,0.0,33.0,2.0,25.0,2.0,3.0,0,110
3572288,1.642288e+09,25,0.0,33.0,2.0,25.0,2.0,3.0,0,110
3572289,1.642288e+09,25,0.0,33.0,2.0,25.0,2.0,3.0,0,110
3572290,1.642288e+09,25,0.0,33.0,2.0,25.0,2.0,3.0,0,110


In [36]:
from sklearn import preprocessing

x = df_subset.drop(['M_WEATHER', 'TIMESTAMP'], axis=1).values
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)

df_norm = pd.DataFrame(x_scaled, columns=df_subset.drop(['M_WEATHER', 'TIMESTAMP'], axis=1).columns.tolist())
df_norm['TIMESTAMP'] = pd.to_datetime(df_subset['TIMESTAMP'], unit='s', dayfirst=True).reset_index(drop=True)
df_norm['M_WEATHER'] = df_subset['M_WEATHER'].reset_index(drop=True)
df_norm = df_norm.set_index('TIMESTAMP')
df_norm

,M_AIR_TEMPERATURE,M_WEATHER_FORECAST_SAMPLES_M_WEATHER,M_WEATHER_FORECAST_SAMPLES_M_TRACK_TEMPERATURE,M_TRACK_TEMPERATURE_CHANGE,M_WEATHER_FORECAST_SAMPLES_M_AIR_TEMPERATURE,M_AIR_TEMPERATURE_CHANGE,M_RAIN_PERCENTAGE,M_AI_DIFFICULTY,M_WEATHER
TIMESTAMP,,,,,,,,,
2022-01-21 00:16:50,0.666667,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0
2022-01-21 00:16:50,0.666667,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0
2022-01-21 00:16:50,0.666667,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0
2022-01-21 00:16:50,0.666667,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0
2022-01-21 00:16:50,0.666667,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0
...,...,...,...,...,...,...,...,...,...
2022-01-15 23:14:51,0.666667,0.0,0.8125,1.0,0.666667,1.0,0.021739,1.0,0
2022-01-15 23:14:51,0.666667,0.0,0.8125,1.0,0.666667,1.0,0.021739,1.0,0
2022-01-15 23:14:51,0.666667,0.0,0.8125,1.0,0.666667,1.0,0.021739,1.0,0


In [37]:
df_norm['M_WEATHER'] = df_norm['M_WEATHER'].apply(lambda x: str(x))
df_norm

,M_AIR_TEMPERATURE,M_WEATHER_FORECAST_SAMPLES_M_WEATHER,M_WEATHER_FORECAST_SAMPLES_M_TRACK_TEMPERATURE,M_TRACK_TEMPERATURE_CHANGE,M_WEATHER_FORECAST_SAMPLES_M_AIR_TEMPERATURE,M_AIR_TEMPERATURE_CHANGE,M_RAIN_PERCENTAGE,M_AI_DIFFICULTY,M_WEATHER
TIMESTAMP,,,,,,,,,
2022-01-21 00:16:50,0.666667,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0
2022-01-21 00:16:50,0.666667,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0
2022-01-21 00:16:50,0.666667,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0
2022-01-21 00:16:50,0.666667,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0
2022-01-21 00:16:50,0.666667,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0
...,...,...,...,...,...,...,...,...,...
2022-01-15 23:14:51,0.666667,0.0,0.8125,1.0,0.666667,1.0,0.021739,1.0,0
2022-01-15 23:14:51,0.666667,0.0,0.8125,1.0,0.666667,1.0,0.021739,1.0,0
2022-01-15 23:14:51,0.666667,0.0,0.8125,1.0,0.666667,1.0,0.021739,1.0,0


In [38]:
df_norm.dropna(inplace=True)

In [39]:
df = df_norm
Y = df['M_WEATHER']
X = df.drop(columns=['M_WEATHER'],inplace=False)

In [40]:
from sklearn.model_selection import train_test_split
X, X_test, y, y_test = train_test_split(X,Y,test_size = 0.2,train_size = 0.8,random_state = 42)
X_train, X_cv, y_train, y_cv = train_test_split(X,y,test_size = 0.25,train_size = 0.75,random_state = 42)

In [41]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(class_weight = 'balanced')
model.fit(X_train,y_train)

RandomForestClassifier(class_weight='balanced')

In [42]:
from numpy import mean
from numpy import std
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(model, X_cv, y_cv, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
# report performance
print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

Accuracy: 0.982 (0.001)


In [43]:
from sklearn.metrics import accuracy_score
predictions = model.predict(X_test)
accuracy = accuracy_score(y_test,predictions)
print(accuracy)

0.9818049208184294


# Pickling the Model

In [45]:
filename = "finalized_randfor_model.sav"
pickle.dump(model, (open(filename, "wb")))